# Assignment 4

Approximate time needed: 6-7 hours
    
## DLT and Epipolar Lines

The goal of this assignment is to aid your understanding of the Direct Linear Transform as well as Epipolar geometry.

For the first part of the assignment you will find correspondences between 2D-3D points and estimate the P Matrix. You will then be required to estimate Camera Parameters from this P matrix.

The second part of this assignment will require you to construct epipolar lines on two corresponding images.

The third part of this assignment will require you to use camera intrinsics to estimate the bounding box of a car.

---
### 4.1 Generating Correspondences
The first step to perform DLT is to generate correspondences. The cell below opens a new window. Clicking anywhere on the image should give you the pixel location of the image. Once you're done clicking, close the image window. The cell after displays the points you have clicked.

After generating pixel locations, you have to generate the corresponding world points. You have the freedom to chose which point you want as origin. You can consider each side as 4 units.

Ensure that the ordering of your points matches the ordering of image points. Keep track of the sequence of corners clicked.

Now that we have the correspondences, we can estimate the P matrix. Implement the DLT function that returns the P matrix

Quick recap: x_i = P * X_i where (x_i, X_i) is the ith correspondence, and P is a 3x4 dimensional matrix. P is further decomposed as P = K * [ R | T].


### Theory

- Explain how DLT is implemented and show why it works (proof)
- When does DLT fail?

## Analysing P

The next part of this question is to analyse how good our estimate is. Report the reprojection error of your P matrix

Given the P matrix, we can estimate K, R and T. Write a function that returns K,R,T given P.
Note: 
P = K * [R | T] where K is a 3x3 matrix and [R | T], P are 3x4 matrices

#### Verifying the R, K, T Values

Use `np.isclose` to verify the matrix

**Reproject the world points with P matrix you have obtained via DLT and visualize**

### Zhangs method

For this task, use the inbuilt Opencv function to estimate the K matrix of a camera. Use the checkerboard images 5456-5470 in `q1/zhangs` folder for this task. Familiarize yourself with the different parameters for this method.

**Reference**: https://docs.opencv.org/3.4/dc/dbb/tutorial_py_calibration.html

---
### 4.2 Epipolar lines
#### Task 1

For this task, you have been given two images of the same scene taken from different view-points. You should first estimate the fundamental matrix from these two images.

<img src="./q2/img1.jpg" alt="image 1" width="400"/>
<img src="./q2/img2.jpg" alt="image 2" width="400"/>


Recall that given a point in one image, it's corresponding location in the other image can be found to along a line viz. the epipolar line. The task given to you is to draw the epipolar lines in the second image for each given point in the first image. You have to repeat this for the other image as well. Draw epipolar lines on the first image for the corresponding points in the second image.

The convention used for F is $x'^{T}Fx$ where $x'$ is the location of the point in the second image. For this question you will need to compute the F matrix on your own without using inbuilt functions. 

In [2]:
import numpy as np
import matplotlib.cm as cm
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib tk

def get_points_byclick(file:str, points_arr:list):
    fig = plt.figure(figsize=(20, 30))

    img = mpimg.imread(file)


    def onclick(event):
        ix, iy = event.xdata, event.ydata
        points_arr.append([ix, iy])


    cid = fig.canvas.mpl_connect('button_press_event', onclick)

    imgplot = plt.imshow(img)
    plt.show()



In [3]:
%matplotlib inline
colors = np.random.rand(10)
def display_points_on_img(file:str, image_points):
    N = len(image_points)
    image_points = np.array(image_points)
    fig = plt.figure(figsize=(10, 15))

    img = mpimg.imread(file)
    imgplot = plt.imshow(img)

    
    area = (15 * np.ones(N))**2

    plt.scatter(image_points[:, 0], image_points[:, 1], c=colors[:N], s=area)
    plt.axis('off')
    plt.show()


In [4]:
##### init data (does not work, separate into blocks)
# %matplotlib tk 
# points_1 = []
# get_points_byclick('q2/img1.jpg', points_1)
# input()
# points_2 = []
# get_points_byclick('q2/img2.jpg', points_2)
# input()
# %matplotlib inline
# display_points_on_img('q2/img1.jpg', points_1)
# display_points_on_img('q2/img2.jpg', points_2)
# with open('data.txt', 'w+') as f:
#     f.write(str(points_1) + '\n')
#     f.write(str(points_2) + '\n')


In [5]:
##### data prerecorded
points_1 = np.array( 
    [[371.2357565035582, 96.69271440633668], [491.8581681476417, 105.37752804471063], [610.5506212054199, 134.3269068392907], [721.5232399179765, 172.92607856539746], [
        666.5194202082746, 378.4666680069157], [650.1147722246792, 469.1747215632665], [426.23957621326036, 516.4587069277472], [324.91675043223023, 318.63795183145027]]
)

points_2 = np.array( 
    [[385.71044590084824, 106.34250733786337], [469.66364440513036, 72.56823207751995], [577.7413252382291, 49.40872904185596], [701.2586747617706, 37.82897752402391], [
        639.4999999999998, 255.91429777652684], [620.2004141369466, 346.62235133287766], [420.4497004543443, 434.43546700977043], [354.8311085199629, 295.4784487957862]]
)


Points selected are:  
<img src="./q2_img1.png" alt="image 1" width="400"/>
<img src="./q2_img2.png" alt="image 2" width="400"/>

In [10]:
def linear_sol(points1, points2):
    def vectorize_point(x, xd):
        x1, y1 = x
        x1d, y1d = xd
        return [ 
            x1d*x1,
            x1d*y1, 
            x1d, 
            y1d*x1,
            y1d*y1, 
            y1d, 
            x1, 
            y1, 
            1
        ]
    
    A = np.array([],dtype=np.float64)
    for x,xd in zip(points1, points2):
        A = np.concatenate((A, vectorize_point(x,xd)))
    A = A.reshape(-1, 9)

    u,d,v = np.linalg.svd(A)
    return v[:, -1].reshape(3,3)


def contraint_enforcement(f):
    u,d,v = np.linalg.svd(f)
    newd = np.diag([d[0], d[1], 0])

    return u@newd@v




In [12]:
f = linear_sol(points_1, points_2)
f_hat = contraint_enforcement(f)

print(f_hat)


[[-3.70187938e-09  3.97651555e-08  7.17021767e-06]
 [ 3.29706320e-05  2.54445804e-03 -3.15540902e-03]
 [-5.47790609e-04  2.96993985e-03  9.99987224e-01]]


**Provide a clear explanation on your approach for both this task and the next**

#### Task 2
Report the location of the epipoles. Mathematically find the epipoles and verify that the lines intersect at the epipole.

____
### 4.3 Drawing a bounding box around a car and localizing it in the camera frame
You’ve been provided with an image, also taken from a self-driving car, that shows another car in front. The camera has been placed on top of the car, 1.65 m from the ground, and assume the image plane is perfectly perpendicular to the ground. K is provided to you. Your task is to draw a 3D-bounding box around the car in front as shown. Your approach should be to place eight points in the 3D world such that they surround all the corners of the car, then project them onto the image, and connect the projected image points using lines. You might have to apply a small 5° rotation about the vertical axis to align the box perfectly. Rough dimensions of the car - h: 1.38 m, w: 1.51, l: 4.10. (Hint: Fix a point on the ground as your world origin.). Also estimate the approximate translation vector to the mid-point of the two rear wheels of the car in the camera frame.

![Kitti car](./q3/image.png)